In [1]:
import numpy 

animal = 'MD594'
with open(f'{animal}_annotations.npy', 'rb') as file:
    volume = numpy.load(file)

In [2]:
from cloudvolume import CloudVolume

cloudpath = f'file:///home/sherry/Projects/pipeline_utility/Litao/{animal}_annotations'
info = CloudVolume.create_new_info(
    num_channels = 1,
    layer_type = 'segmentation',
    data_type = str(volume.dtype), # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = [14464, 14464, 20000], # Voxel scaling, units are in nanometers
    voxel_offset = [0, 0, 0], # x,y,z offset in voxels from the origin
    chunk_size = [128, 128, 16], # units are voxels
    volume_size = volume.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False)
vol.commit_info()
vol[:, :, :] = volume[:, :, :]

Uploading: 100%|██████████| 2016/2016 [00:11<00:00, 45.75it/s]

In [3]:
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc

tq = LocalTaskQueue(parallel=4)
tasks = tc.create_downsampling_tasks(cloudpath)
tq.insert(tasks)
tasks = tc.create_meshing_tasks(cloudpath, mip=0, compress=False)
tq.insert(tasks)
tasks = tc.create_mesh_manifest_tasks(cloudpath)
tq.insert(tasks)
tq.execute(progress=True)


Tasks:   0%|          | 0/28 [00:00<?, ?it/s]

Volume Bounds:  Bbox([0, 0, 0],[1468, 728, 433], dtype=int32)
Selected ROI:   Bbox([0, 0, 0],[1468, 728, 433], dtype=int32)




Downloading:   0%|          | 0/72 [00:00<?, ?it/s]

Downloading:   0%|          | 0/72 [00:00<?, ?it/s]

Downloading:   0%|          | 0/72 [00:00<?, ?it/s]

Downloading:   0%|          | 0/72 [00:00<?, ?it/s]

Downloading:  43%|████▎     | 31/72 [00:00<00:00, 244.77it/s]

Downloading:  54%|█████▍    | 39/72 [00:00<00:00, 283.90it/s]

Downloading:  54%|█████▍    | 39/72 [00:00<00:00, 302.41it/s]

Downloading:  54%|█████▍    | 39/72 [00:00<00:00, 302.41it/s]

Downloading:  68%|██████▊   | 49/72 [00:00<00:00, 302.41it/s]

Downloading:  76%|███████▋  | 55/72 [00:00<00:00, 302.41it/s]

Downloading:  94%|█████████▍| 68/72 [00:00<00:00, 268.43it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 268.43it/s]

Downloading:  93%|█████████▎| 67/72 [00:00<00:00, 302.41it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 302.41it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 302.41it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 268.43it/s]

Downloading: 100%|██████████| 

Uploading: 100%|██████████| 18/18 [00:00<00:00, 119.01it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 119.01it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 119.01it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 119.01it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 119.01it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 119.01it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 119.01it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 119.01it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 119.01it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 119.01it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 56.10it/s] 


Uploading:   0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 55.04it/s] 


Uploading:   0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 100%|██████████| 6/6 [00:00<00:00, 134.17it/s]


Downloading:   0%|          | 0/72 [00:00<?, ?it/s]

Downloading:   0%|          | 0/72 [00:00<?, ?it/s]



Downloading:   0%|          | 0/72 [00:00<?, ?it/s]

Uploading: 100%|██████████| 6/6 [00:00<00:00, 83.81it/s]A


Downloading:   0%|          | 0/72 [00:00<?, ?it/s]

Downloading:  60%|█████▉    | 43/72 [00:00<00:00, 371.73it/s]

Downloading:  68%|██████▊   | 49/72 [00:00<00:00, 401.89it/s]

Downloading:  71%|███████   | 51/72 [00:00<00:00, 401.89it/s]

Downloading:  74%|███████▎  | 53/72 [00:00<00:00, 401.89it/s]

Downloading:  81%|████████  | 58/72 [00:00<00:00, 401.89it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 417.26it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 417.26it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 417.26it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 417.26it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 401.89it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 401.89it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 417.26it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 417.26it/s]

Downloading: 10

Uploading: 100%|██████████| 18/18 [00:00<00:00, 116.40it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 116.40it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 116.40it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 116.40it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 113.41it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 116.40it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 113.41it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 113.41it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 113.41it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 113.41it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 113.41it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 113.41it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 113.41it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 113.41it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 113.41it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 113.41it/s]

Uploading: 100%|████████

Downloading: 100%|██████████| 72/72 [00:00<00:00, 345.04it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 388.95it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 388.95it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 388.95it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 388.95it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 388.95it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 345.04it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 345.04it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 345.04it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 345.04it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 121.36it/s]


Uploading:   0%|          | 0/18 [00:00<?, ?it/s]

Uploading:   0%|          | 0/18 [00:00<?, ?it/s]

Uploading:   0%|          | 0/18 [00:00<?, ?it/s]

Uploading:   0%|          | 0/18 [00:00<?, ?it/s]

Uploading: 100%|██████████| 18/18 [00:00<00:00, 132.35it/s]

Uploading: 100%|██████████| 18/18 [00:00<

Downloading: 100%|██████████| 72/72 [00:00<00:00, 438.03it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 438.03it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 420.24it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 438.03it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 420.24it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 438.03it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 438.03it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 438.03it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 438.03it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 420.24it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 442.05it/s]

Downloading:  86%|████████▌ | 62/72 [00:00<00:00, 465.07it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 420.24it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 442.05it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 442.05it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 465.0

Downloading: 100%|██████████| 72/72 [00:00<00:00, 241.14it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 241.14it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 241.14it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 411.75it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 241.14it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 241.14it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 411.75it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 241.14it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 411.75it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 411.75it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 411.75it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 411.75it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 241.14it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 241.14it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 241.14it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 241.1

Downloading: 100%|██████████| 72/72 [00:00<00:00, 432.07it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 432.07it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 432.07it/s]

Downloading:  86%|████████▌ | 62/72 [00:00<00:00, 471.90it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 431.95it/s]

Downloading:  90%|█████████ | 65/72 [00:00<00:00, 471.90it/s]

Downloading:  94%|█████████▍| 68/72 [00:00<00:00, 471.90it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 431.95it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 432.07it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 432.07it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 431.95it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 431.95it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 431.95it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 431.95it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 471.90it/s]

Downloading: 100%|██████████| 72/72 [00:00<00:00, 432.0